In [1]:
from data import DataGenerator

data = DataGenerator()

In [2]:
train = data.add_negatives(data.train, n_samples=4)

X_train = data.get_features(train)
y_train = data.get_target(train)

In [3]:
from models import Model
model = Model(data.num_users, data.num_movies, latent_dim=8)


In [4]:
from tensorflow import config
def find_batch_size():
    if len(config.list_physical_devices('GPU'))>0:
        return 1024
    else:
        return 64
batch_size = find_batch_size()


In [5]:
test = data.add_negatives(data.test, n_samples=100)

X_test = data.get_features(test)
y_test = data.get_target(test)


In [6]:
model.GMF()
model.compile_model()

2021-10-24 23:33:26.317368: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/franciscocancedda/Documents/Scool/NFPCF/venv/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [7]:
epochs = 3

model.train_model(X_train, y_train, test, X_test, batch_size, epochs)
# mlp.train_model(X_train, y_train, test, X_test, batch_size, epochs)
# ncf.train_model(X_train, y_train, test, X_test, batch_size, epochs)

2021-10-24 23:33:52.458618: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
15912/15912 [==============================] - 13s 764us/step - loss: 0.1219

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [73]:
import pandas as pd
def features():

    df = pd.read_csv('MovieLens/users.dat',
                     sep='::',
                     header=None,
                     names=['uid', 'gender', 'age', 'job', 'zip'],
                     engine='python')
    df.drop(columns=['uid'], inplace=True)
    df.index.rename('uid', inplace=True)
    df.gender = pd.get_dummies(df.gender, drop_first=True)  # 0:F, 1:M
    return df

df = features()
df.head()

,gender,age,job,zip
uid,,,,
0,0,1,10,48067
1,1,56,16,70072
2,1,25,15,55117
3,1,45,7,02460
4,1,25,20,55455


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  6040 non-null   uint8 
 1   age     6040 non-null   int64 
 2   job     6040 non-null   int64 
 3   zip     6040 non-null   object
dtypes: int64(2), object(1), uint8(1)
memory usage: 147.6+ KB


In [39]:
# from time import time

,gender,age,job,zip
uid,,,,
1,1,56,16,70072
2,1,25,15,55117
3,1,45,7,02460
4,1,25,20,55455
5,0,50,9,55117


In [45]:
jobs = {
0:  "other",  # drop
1:  "academic/educator",
2:  "artist",
3:  "clerical/admin",
4:  "college/grad student",
5:  "customer service",
6:  "doctor/health care",
7:  "executive/managerial",
8:  "farmer",
9:  "homemaker",
10:  "K-12 student",  # drop
11:  "lawyer",
12:  "programmer",
13:  "retired",  # drop
14:  "sales/marketing",
15:  "scientist",
16:  "self-employed",
17:  "technician/engineer",
18:  "tradesman/craftsman",
19:  "unemployed",  # drop
20:  "writer",
}

0: dropped other
academic/educator
artist
clerical/admin
college/grad student
customer service
doctor/health care
executive/managerial
farmer
homemaker
10: dropped K-12 student
lawyer
programmer
13: dropped retired
sales/marketing
scientist
self-employed
technician/engineer
tradesman/craftsman
19: dropped unemployed
writer


17

In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3479 entries, 1 to 6039
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  3479 non-null   uint8 
 1   age     3479 non-null   int64 
 2   job     3479 non-null   int64 
 3   zip     3479 non-null   object
dtypes: int64(2), object(1), uint8(1)
memory usage: 112.1+ KB


In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1441 entries, 5 to 6036
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  1441 non-null   uint8 
 1   age     1441 non-null   int64 
 2   job     1441 non-null   int64 
 3   zip     1441 non-null   object
dtypes: int64(2), object(1), uint8(1)
memory usage: 46.4+ KB


In [78]:
complete = pd.merge(data.df, df, how='inner', on='uid')


In [81]:
complete.drop(columns=['date', 'latest', 'zip'], inplace=True)

In [82]:
complete.head()



,uid,mid,rating,gender,age,job
0,0,1192,1,0,1,10
1,0,660,1,0,1,10
2,0,913,1,0,1,10
3,0,3407,1,0,1,10
4,0,2354,1,0,1,10


In [83]:
complete.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   uid     1000209 non-null  int64
 1   mid     1000209 non-null  int64
 2   rating  1000209 non-null  int8 
 3   gender  1000209 non-null  uint8
 4   age     1000209 non-null  int64
 5   job     1000209 non-null  int64
dtypes: int64(4), int8(1), uint8(1)
memory usage: 40.1 MB


In [86]:
drop = [0, 10, 13, 19]

clean = complete[~complete['job'].isin(drop)]

0: dropped other
academic/educator
artist
clerical/admin
college/grad student
customer service
doctor/health care
executive/managerial
farmer
homemaker
10: dropped K-12 student
lawyer
programmer
13: dropped retired
sales/marketing
scientist
self-employed
technician/engineer
tradesman/craftsman
19: dropped unemployed
writer


In [87]:
import numpy as np

msk = np.random.rand(len(clean)) < 0.7

train = clean[msk]
test = clean[~msk]

clean.info()  # 1476+3444 = 4920


<class 'pandas.core.frame.DataFrame'>
Int64Index: 817762 entries, 53 to 1000208
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   uid     817762 non-null  int64
 1   mid     817762 non-null  int64
 2   rating  817762 non-null  int8 
 3   gender  817762 non-null  uint8
 4   age     817762 non-null  int64
 5   job     817762 non-null  int64
dtypes: int64(4), int8(1), uint8(1)
memory usage: 32.8 MB


In [90]:
num_users = clean.uid.nunique()
num_movies = clean.mid.nunique()
num_jobs = clean.job.nunique()

In [105]:
embeds = model.gmf_user_embedding.get_weights()[0]


In [108]:
gender_embed = np.zeros((2,embeds.shape[1]))
num_users_x_group = np.zeros((2, 1))


In [122]:
# S = 0 indicates male and S = 1 indicates female

for i in range(train.shape[0]):
    u = train['uid'].iloc[i]
    if train['gender'].iloc[i] == 0:
        gender_embed[0] +=  embeds[u]
        num_users_x_group[0] += 1.0
    else:
        gender_embed[1] +=  embeds[u]
        gender_embed[1] += 1.0
        num_users_x_group[1] += 1.0

In [123]:
gg = gender_embed / num_users_x_group

In [124]:
vBias= gg[1].reshape((1,-1)) - gg[0].reshape((1,-1))
vBias = vBias / np.linalg.norm(vBias,axis=1,keepdims=1)

In [125]:
vBias

array([[0.4267244 , 0.04773208, 0.05010729, 0.43557079, 0.44262408,
        0.44283661, 0.44027303, 0.19373847]])

In [129]:
# linear projection: u - <u,v_b>v_b

for i in range(len(clean)):
    u = clean['uid'].iloc[i]
    embeds[u] = embeds[u] - (np.inner(embeds[u].reshape(1,-1),vBias)[0][0])*vBias

In [130]:
embeds

array([[ 0.90561336, -0.8011142 , -0.880663  , ...,  0.83978665,
         0.8457549 , -0.29267547],
       [ 0.22851142, -1.4139832 , -1.3438339 , ...,  0.20110954,
         0.23578653, -1.5130116 ],
       [-0.01704877, -1.2103463 , -1.2383071 , ...,  0.15906374,
         0.11616315, -0.65695375],
       ...,
       [ 0.08523539, -0.6899795 , -0.6887264 , ...,  0.11674011,
         0.1026208 , -0.5665068 ],
       [ 1.0493549 , -1.3370248 , -1.2041548 , ...,  1.0906494 ,
         1.2938513 , -0.85203683],
       [-0.03722361, -1.7358453 , -1.8662591 , ...,  0.20659442,
         0.30186534, -1.0799204 ]], dtype=float32)

In [131]:
def criterionHinge(epsilonClass, epsilonBase):
    zeroTerm = 0
    return np.max(zeroTerm, (epsilonClass-epsilonBase))

In [132]:
import tensorflow as tf
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def computeEDF(protectedAttributes,predictions,numClasses,item_input,device):
    # compute counts and probabilities
    S = np.unique(protectedAttributes) # number of gender: male = 0; female = 1
    countsClassOne = torch.zeros((numClasses,len(S)),dtype=torch.float).to(device) #each entry corresponds to an intersection, arrays sized by largest number of values
    countsTotal = torch.zeros((numClasses,len(S)),dtype=torch.float).to(device)

    concentrationParameter = 1.0
    dirichletAlpha = concentrationParameter/numClasses

    for i in range(len(predictions)):
        countsTotal[item_input[i],protectedAttributes[i]] = countsTotal[item_input[i],protectedAttributes[i]] + 1.0
        countsClassOne[item_input[i],protectedAttributes[i]] = countsClassOne[item_input[i],protectedAttributes[i]] + predictions[i]

    #probabilitiesClassOne = countsClassOne/countsTotal
    probabilitiesForDFSmoothed = (countsClassOne + dirichletAlpha) /(countsTotal + concentrationParameter)
    avg_epsilon = differentialFairnessMultiClass(probabilitiesForDFSmoothed,numClasses,device)
    return avg_epsilon

def differentialFairnessMultiClass(probabilitiesOfPositive,numClasses,device):
    # input: probabilitiesOfPositive = positive p(y|S) from ML algorithm
    # output: epsilon = differential fairness measure
    epsilonPerClass = torch.zeros(len(probabilitiesOfPositive),dtype=torch.float).to(device)
    for c in  range(len(probabilitiesOfPositive)):
        epsilon = torch.tensor(0.0).to(device) # initialization of DF
        for i in  range(len(probabilitiesOfPositive[c])):
            for j in range(len(probabilitiesOfPositive[c])):
                if i == j:
                    continue
                else:
                    epsilon = torch.max(epsilon,torch.abs(torch.log(probabilitiesOfPositive[c,i])-torch.log(probabilitiesOfPositive[c,j]))) # ratio of probabilities of positive outcome
        #                    epsilon = torch.max(epsilon,torch.abs((torch.log(1-probabilitiesOfPositive[c,i]))-(torch.log(1-probabilitiesOfPositive[c,j])))) # ratio of probabilities of negative outcome
        epsilonPerClass[c] = epsilon # overall DF of the algorithm
    avg_epsilon = torch.mean(epsilonPerClass)
    return avg_epsilon

In [157]:
from tensorflow.keras.optimizers import Adam
epsilonBase= 0
fairness_thres = .1
# FIXME model params weight decay
optimizer = Adam(model.parameters(), lr=.001, weight_decay=1e-6)
# for i in range(epochs)
for batch in np.array_split(train, batch_size):
    batch = data.add_negatives(batch[['uid', 'mid', 'rating']], n_samples=4)
    # print(batch[(batch.uid==4)&(batch.rating==0)])
    predictions = model.model.predict([batch.uid.to_numpy(), batch.mid.to_numpy()])
    loss1 = bce(predictions[:,0], batch.rating.to_numpy())
    print(loss1)
    all_predictions = model.model.predict([train.uid.to_numpy(), train.mid.to_numpy()])
    avg_eps = computeEDF()
    loss2  = criterionHinge(avg_eps, epsilonBase)
    loss = loss1 + fairness_thres*loss2

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    break


InvalidArgumentError: Value for attr 'T' of int8 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Exp}}; Op<name=Exp; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Exp]